### Time Loop

In [10]:
from ipynb.fs.full.Background_Functions import *
from ipynb.fs.full._06_Solution_Algorithm import *


import math
import numpy as np

#### Final notebook output: 






In [11]:
nx, ny

(40, 4)

In [12]:
## Define required variables

tf = 2           #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/2          #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)



t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

#rhs vector
b_x = np.zeros([(ny+2)*(nx+2),1])
b_y = np.zeros([(ny+2)*(nx+2),1])

In [13]:
from scipy.sparse.linalg import spsolve


In [14]:
tolerance = 1e-6
b_x = np.zeros([(ny+2)*(nx+2),1])
b_y = np.zeros([(ny+2)*(nx+2),1])

U_new = np.zeros([(ny+2)*(nx+2),2])
U_old = np.zeros([(ny+2)*(nx+2),2])
U_old_old = np.zeros([(ny+2)*(nx+2),2])

In [15]:
import pandas as pd

In [16]:
## Time loop

!rm -r savedVariables
! mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100000
    fac = 10**math.ceil((math.log((A("x").a_P), 10)))

    # Make directory

    ! mkdir savedVariables/{time}

    residual_array = np.array([])
    
    while True:
    # for iteration in [1]:
        
        U_previous = U_new #store latest guess values    # change_u new to U

        A_x = A("x").createMatrix()        
        A_x, b_x = boundary_conditions(A_x, b_x, U_previous, U_old, U_old_old, "x")

        pd.DataFrame(A_x).to_csv("savedVariables/" + "A_x" + ".csv", index=False, header=None)

        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A("y").createMatrix()
        A_y, b_y = boundary_conditions(A_y, b_y, U_previous, U_old, U_old_old, "y")
        
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)
        
        # ## Update U.New with new x and y displacements

        U_new = np.vstack((Ux, Uy)).T
       
        ## Calculate the residual of each iteration
        
        normFactor = np.max(U_new)
        residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor
        tolerance = 1e-6

        residual_array = np.append(residual_array, residual)

        # print(icorr)
        # print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, time = {:.5f}".format(icorr, residual, normFactor, time))

    
        ##print out residual of each iteration
        if icorr%100 == 0 :
            moving_average = np.mean(residual_array[len(residual_array)-100 :len(residual_array) ])
            print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(icorr, residual, normFactor, moving_average, time))
            print(tolerance - residual)


        ## Convergence check
        
        if residual < tolerance:

            print(residual)
            
            print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}, moving average = {:.20f}, time = {:.5f}".format(icorr, residual, normFactor, moving_average, time))

            break
            
        elif icorr > maxcorr:
            
            break            
        
        icorr = icorr + 1
    
    ## Update variables
    
    U_old_old = U_old
    U_old = U_new
    
    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = U_new[:,0].reshape(ny+2, nx+2)
    u_reshape_y = U_new[:,1].reshape(ny+2, nx+2)
    
    # #Boundary conditions for u matrix
    U = np.zeros((ny+2,nx+2))
    V = np.zeros((ny+2,nx+2))
 
 
    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y 

    #Modulus of displacement

    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
   ### Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
fig.show()   

print("U graph")
fig = go.Figure(data=[go.Surface(z=U, x=dispx, y=dispy)])

fig.show()   

print("V graph")
fig = go.Figure(data=[go.Surface(z=V, x=dispx, y=dispy)])

fig.show()


Time = 0.0
icorr = 100, Residual = 0.00861212273732550604, normFactor = 0.00003249705114586239, moving average = 4730760749301.85644531250000000000, time = 0.00000
-0.008611122737325507
icorr = 200, Residual = 0.00391950111409105836, normFactor = 0.00006476581518589251, moving average = 0.07861821722827591619, time = 0.00000
-0.003918501114091058
icorr = 300, Residual = 0.00241552941698397168, normFactor = 0.00009594279699085700, moving average = 0.04010187889655321414, time = 0.00000
-0.0024145294169839716
icorr = 400, Residual = 0.00169048478586489434, normFactor = 0.00012527358440423769, moving average = 0.02577807253600772341, time = 0.00000
-0.0016894847858648944
icorr = 500, Residual = 0.00126925296165225697, normFactor = 0.00015245681107656784, moving average = 0.01848121474168253411, time = 0.00000
-0.001268252961652257
icorr = 600, Residual = 0.00099621193662743175, normFactor = 0.00017746276984364199, moving average = 0.01410911545090359244, time = 0.00000
-0.000995211936627

In [ ]:
A("x").a_S*2/(Sfy)

6153846153846.153

In [ ]:
Sfy

0.05

#### Final notebook output: 




